In [1]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 31.4 MB/s 
     |████████████████████████████████| 1.6 MB 58.5 MB/s 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.merge import concatenate
from sklearn.model_selection import train_test_split


physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

GPUs Available:  1


In [3]:
dfs = []
for i in range(0, 13, 2):
    dfs.append(pd.read_excel("small.xlsx", sheet_name=i).iloc[3:, :])

In [11]:
df = pd.concat(dfs)
df = df.iloc[:, [1, 2, 3, 5, 6, 7, 8, 9, 10, 18, 20, 24, 27]]
df.shape

(1640, 13)

In [12]:
df.dropna(inplace = True)
df.shape

(684, 13)

In [28]:
df = df.apply(pd.to_numeric)

In [29]:
X = df.iloc[:,[0, 1, 3, 4, 5]].copy()
Y = df.iloc[:,[2, 6, 7, 8, 9, 10, 11, 12]].copy()

In [30]:
X.shape

(684, 5)

In [31]:
from sklearn import preprocessing
X = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)

In [32]:
x_scaled.shape

(684, 5)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(x_scaled, Y, test_size=0.2, random_state=42)

In [34]:
def model_builder(hp):
    input = Input(shape=(X_train.shape[1]), name="Input")

    x = Dense(256, activation="ReLU")(input)
    x = Dropout(0.2)(x)


    x = Dense(256, activation="ReLU")(x)
    x = Dropout(0.2)(x)


    x = Dense(256, activation="ReLU")(x)
    x = Dropout(0.2)(x)

    x = Dense(256, activation="ReLU")(x)

    # x = Dense(256, activation="ReLU")(x)


    output = Dense(y_train.shape[1])(x)

    model = Model(inputs=input, outputs=output)
    


    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])

    hp_decay = hp.Choice('decay', values=[0.99, 0.95, 0.9])

    opt = tf.keras.optimizers.Adam(
        learning_rate=hp_learning_rate,
        decay=hp_decay
    )

    model.compile(optimizer=opt,
                loss='mse',
                metrics=['mse'])



    # print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [35]:
%rm -r /content/drive/MyDrive/IPCO/kat/IPCO

In [36]:
tuner = kt.Hyperband(model_builder,
                     objective='val_mse',
                     max_epochs=100,
                     factor=16,
                     directory='/kat',
                     project_name='IPCO')

In [37]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=4)

In [38]:
tuner.search(X_train, y_train, validation_data=(X_test, y_test), callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters()[0]

Trial 12 Complete [00h 00m 02s]
val_mse: 27536276.0

Best val_mse So Far: 27374444.0
Total elapsed time: 00h 00m 27s


In [ ]:
best_hps.values

In [ ]:
best_model = tuner.get_best_models()[0]

In [ ]:
history = best_model.fit(x=X_train, y=y_train, batch_size=64, epochs=100,
                    verbose=1, validation_data=[X_test, y_test])